In [1]:
import pandas as pd
import pickle
import numpy as np
from scipy import stats
from sklearn import preprocessing, decomposition, manifold
from sklearn import random_projection
import operator

## 4. Feature Engineering

In [2]:
# read dt_all_eng
dt_all_eng = pd.read_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_encoded.csv")
# read cols_raw
cols_raw = pickle.load(open("../../data/Mercedes_Benz_Greener_Manufacturing/data/cols_raw.pkl", "rb"))
# IDs
IDs_train = pickle.load(open("../../data/Mercedes_Benz_Greener_Manufacturing/data/IDs_train.pkl", "rb"))
IDs_test = pickle.load(open("../../data/Mercedes_Benz_Greener_Manufacturing/data/IDs_test.pkl", "rb"))

In [3]:
dt_all_eng.shape

(8418, 402)

### 4.1 Outlier marker

In [4]:
def getOutlierMarker(dt, cols_type, id_outlier = 1770):
    
    if cols_type == "Cat":
        cols = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']
    elif cols_type == "Bin":
        cols = cols_raw
    elif cols_type == "All":
        cols = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'] + list(cols_raw)
    else:
        cols = [cols_type]
    
    # outlier
    dt_outlier = dt[cols].loc[dt["ID"] == id_outlier]
    # calc int_outlierMarker
    dt_outlierMaker = dt_all_eng[["ID"]]
    for col in cols:
        dt_outlierMaker_id_col = pd.DataFrame()
        for id in dt.ID.values:
            is_equal = dt.loc[dt["ID"] == id][col].values == dt_outlier[col].values
            dt_outlierMaker_id_col = pd.concat([dt_outlierMaker_id_col
                                        , pd.DataFrame({"ID": id
                                           , col + "_is_equal": is_equal.astype("int64")})])
        
        dt_outlierMaker = pd.merge(dt_outlierMaker, dt_outlierMaker_id_col, on = "ID")
        
    sum_outlierMaker = dt_outlierMaker.filter(regex = "_is_equal").sum(axis = 1)
    dt["FeatEng_OutlierMaker_" + str(id_outlier) + "_" + cols_type] = sum_outlierMaker
#     for col in dt_outlier.columns.values:
#         for val in dt_outlier[col].values:
#             int_outlierMarker = int_outlierMarker + (dt[col].values == val).astype("int64")
    
#     dt["FeatEng_OutlierMaker_" + str(id_outlier) + "_" + cols_type] = int_outlierMarker
    
    return dt

#### 4.1.1 OutlierMarker_Cat

In [5]:
dt_all_eng = getOutlierMarker(dt_all_eng, cols_type = "Cat")

#### 4.1.2 OutlierMarker_Int

In [6]:
dt_all_eng = getOutlierMarker(dt_all_eng, cols_type = "Bin")

#### 4.1.3 OutlierMarker_All

In [7]:
dt_all_eng = getOutlierMarker(dt_all_eng, cols_type = "All")

#### 4.1.4 OutlierMarker_X0

In [8]:
dt_all_eng = getOutlierMarker(dt_all_eng, cols_type = "X0")

In [9]:
dt_all_eng.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,Encode_Label_X2,Encode_Label_X3,Encode_Label_X4,Encode_Label_X5,Encode_Label_X6,Encode_Label_X8,FeatEng_OutlierMaker_1770_Cat,FeatEng_OutlierMaker_1770_Bin,FeatEng_OutlierMaker_1770_All,FeatEng_OutlierMaker_1770_X0
0,0,130.81,k,v,at,a,d,u,j,o,...,45,1,4,20,10,15,1,271,272,0
1,5761,115.07,o,l,ae,f,d,p,j,o,...,30,6,4,15,10,15,2,293,295,0
2,5883,77.38,bc,v,ac,f,d,p,j,o,...,28,6,4,15,10,15,2,268,270,0
3,2932,108.06,ak,v,ak,f,d,ac,j,o,...,36,6,4,28,10,15,2,299,301,0
4,2849,0.00,s,aa,ay,g,d,ac,j,o,...,50,7,4,28,10,15,1,297,298,0


### 4.2 Sum of binary cols

#### 4.2.1 Sum of all binary cols

In [10]:
sum_binary_all = dt_all_eng[cols_raw].sum(axis = 1)
dt_all_eng["FeatEng_SumBin_All"] = sum_binary_all

#### 4.2.2 Sum of correlation-important binary cols

In [11]:
def corBin_Contin(dt, cols, method = "pointbiserialr"):
    dt_binary_pointbiserialr = pd.DataFrame()
    for col in cols:
        if method == "spearmanr":
            cor_pb = stats.spearmanr(dt[col].values, dt.y.values)
        else:
            cor_pb = stats.pointbiserialr(dt[col].values, dt.y.values)
        dt_binary_pointbiserialr = pd.concat([dt_binary_pointbiserialr
                                             , pd.DataFrame({"Col": col
                                                            , "Cor": np.abs([cor_pb.correlation])
                                                            , "P": [cor_pb.pvalue]})])
    return dt_binary_pointbiserialr.sort_values("Cor", ascending = False)

In [12]:
dt_corBin_Cotin = corBin_Contin(dt_all_eng.loc[dt_all_eng["ID"].isin(IDs_train)], cols_raw)

/usr/local/lib/python3.5/dist-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den
/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in absolute
  # Remove the CWD from sys.path while we load stuff.


In [13]:
dt_corBin_Cotin_spear = corBin_Contin(dt_all_eng.loc[dt_all_eng["ID"].isin(IDs_train)], cols_raw, method = "spearmanr")

/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:3003: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:3004: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
/home/noahhhhhh/Env/deepNoah/lib/python3.5/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in absolute
  # Remove the CWD from sys.path while we load stu

In [14]:
cols_binary_cor_02_10 = dt_corBin_Cotin_spear.loc[dt_corBin_Cotin_spear["Cor"] >= .2]["Col"].values
cols_binary_cor_01_02 = dt_corBin_Cotin_spear.loc[(dt_corBin_Cotin_spear["Cor"] >= .1) & (dt_corBin_Cotin_spear["Cor"] < .2)]["Col"].values
cols_binary_cor_005_01 = dt_corBin_Cotin_spear.loc[(dt_corBin_Cotin_spear["Cor"] >= .05) & (dt_corBin_Cotin_spear["Cor"] < .1)]["Col"].values

In [15]:
cols_binary_important_3 = dt_corBin_Cotin["Col"].values[:3]
cols_binary_important_5 = dt_corBin_Cotin["Col"].values[:5]
cols_binary_important_10 = dt_corBin_Cotin["Col"].values[:10]
cols_binary_important_20 = dt_corBin_Cotin["Col"].values[:20]
cols_binary_important_50 = dt_corBin_Cotin["Col"].values[:50]

In [16]:
sum_binary_important_3 = dt_all_eng[cols_binary_important_3].sum(axis = 1)
sum_binary_important_5 = dt_all_eng[cols_binary_important_5].sum(axis = 1)
sum_binary_important_10 = dt_all_eng[cols_binary_important_10].sum(axis = 1)
sum_binary_important_20 = dt_all_eng[cols_binary_important_20].sum(axis = 1)
sum_binary_important_50 = dt_all_eng[cols_binary_important_50].sum(axis = 1)
sum_binary_important_cor_02_10 = dt_all_eng[cols_binary_cor_02_10].sum(axis = 1)
sum_binary_important_cor_01_02 = dt_all_eng[cols_binary_cor_01_02].sum(axis = 1)
sum_binary_important_cor_005_01 = dt_all_eng[cols_binary_cor_005_01].sum(axis = 1)

In [17]:
# SumBin_All
dt_all_eng.loc[:, "FeatEng_SumBin_All"] = sum_binary_all
# SumBin_Imp_X
dt_all_eng.loc[:, "FeatEng_SumBin_Imp_3"] = sum_binary_important_3
dt_all_eng.loc[:, "FeatEng_SumBin_Imp_5"] = sum_binary_important_5
dt_all_eng.loc[:, "FeatEng_SumBin_Imp_10"] = sum_binary_important_10
dt_all_eng.loc[:, "FeatEng_SumBin_Imp_20"] = sum_binary_important_20
dt_all_eng.loc[:, "FeatEng_SumBin_Imp_50"] = sum_binary_important_50
dt_all_eng.loc[:, "FeatEng_SumBin_Cor_02_10"] = sum_binary_important_cor_02_10
dt_all_eng.loc[:, "FeatEng_SumBin_Cor_01_02"] = sum_binary_important_cor_01_02
dt_all_eng.loc[:, "FeatEng_SumBin_Cor_005_01"] = sum_binary_important_cor_005_01

In [18]:
dt_all_eng.shape

(8418, 415)

### 4.3 Dimension reduction

In [19]:
cols_cat = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']

In [20]:
# # range
# pp_range = preprocessing.MinMaxScaler()
# mx_range = pp_range.fit_transform(dt_all_eng.drop(["ID", "y", "IsTrainTest"], axis = 1))
# dt_all_eng = pd.DataFrame(mx_range, columns = dt_all_eng.drop(["ID", "y", "IsTrainTest"], axis = 1).columns.values)
# dt_all_eng.shape

In [21]:
def featEng_dimRed(method, dt, n_component, name_feature):
        
    if method == "PCA":
        dr = decomposition.PCA(n_components = n_component, random_state = 420)
    elif method == "ICA":
        dr = decomposition.FastICA(n_components = n_component, random_state = 420)
    elif method == "SVD":
        dr = decomposition.TruncatedSVD(n_components = n_component, random_state = 420)
    elif method == "FA":
        dr = decomposition.FactorAnalysis(n_components = n_component, random_state = 420)
    elif method == "TSNE":
        dr = manifold.TSNE(n_components = n_component, random_state = 420)
    elif method == "GRP":
        dr = random_projection.GaussianRandomProjection(n_components = n_component, eps = 0.1, random_state = 420)
    elif method == "SRP":
        dr = random_projection.SparseRandomProjection(n_components = n_component, dense_output = True, random_state = 420)
    
    if name_feature == "Raw_Bin":
        cols = cols_raw
    elif name_feature == "Encoded_Cat":
        cols = dt.filter(regex = "Encode_").columns.values
    elif name_feature == "FeatEng":
        cols = dt.filter(regex = "FeatEng_").columns.values
    elif name_feature == "All":
        cols = dt.drop(["y"] + cols_cat, axis = 1).filter(regex = "^((?!DR).)*$").columns.values
    else:
        cols = name_feature
        
    mx = dr.fit_transform(dt[cols])
    dt = dt.join(pd.DataFrame(mx, columns = ["DR_" + method + "_" + name_feature + "_" + str(i) for i in range(1, n_component + 1)]))
    
    return dt

In [22]:
methods_dr = ["PCA", "ICA", "SVD", "FA", "GRP", "SRP"] # TSNE
for method in methods_dr:
    dt_all_eng = featEng_dimRed(method, dt_all_eng, 10, "Raw_Bin")
    dt_all_eng = featEng_dimRed(method, dt_all_eng, 10, "Encoded_Cat")
    dt_all_eng = featEng_dimRed(method, dt_all_eng, 3, "FeatEng")
    dt_all_eng = featEng_dimRed(method, dt_all_eng, 15, "All")

/usr/local/lib/python3.5/dist-packages/sklearn/decomposition/fastica_.py:116: UserWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn('FastICA did not converge. Consider increasing '


In [23]:
dt_all_eng.shape

(8418, 643)

### 4.5 Feature Interaction

In [24]:
dt_interest = dt_all_eng[cols_raw]
dt_fi_2way = pd.DataFrame()
seen = []
for c1 in cols_raw:
    for c2 in cols_raw:
        if c1 != c2:
            seen.append((c1, c2))
            if (c2, c1) not in seen:
                isTrain = dt_all_eng["ID"].isin(IDs_train)
                val_c1 = dt_interest[c1].values.astype("int64")
                val_c2 = dt_interest[c2].values.astype("int64")
                y_train = dt_all_eng.y.values[isTrain]
                # XOR
                XOR = val_c1 ^ val_c2
                cor_c1 = abs(stats.spearmanr(val_c1[isTrain], y_train).correlation)
                cor_c2 = abs(stats.spearmanr(val_c2[isTrain], y_train).correlation)
                cor_XOR = abs(stats.spearmanr(XOR[isTrain], y_train).correlation)
                # constant
                if cor_XOR > .3:
                    const = 1.1
                elif (cor_XOR > .2) & (cor_XOR <= .3):
                    const = 1.3
                else:
                    const = 1.5
                if cor_XOR > max(cor_c1, cor_c2) * const:
                    dt_fi_2way.loc[:, "FeatEng_FI_2way_XOR_" + c1 + "_" + c2] = XOR

/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:3003: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.5/dist-packages/numpy/lib/function_base.py:3004: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/usr/local/lib/python3.5/dist-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [25]:
dt_all_eng = dt_all_eng.join(dt_fi_2way)

In [26]:
dt_all_eng.shape

(8418, 1361)

### 4.6 Save dt_all_eng

In [27]:
dt_all_eng.to_csv("../../data/Mercedes_Benz_Greener_Manufacturing/data/dt_all_eng.csv", index = False)